In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
import csv
from tqdm import tqdm

# 크롬 옵션 설정 (헤드리스: 화면 없이 실행)
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--window-size=1920,1080')

# 드라이버 로딩
driver = webdriver.Chrome(options=options)

# 크롤링할 페이지 열기
url = "https://page.kakao.com/content/48787313"
driver.get(url)
time.sleep(5)

# results 초기화
results = []

# 수집 예상 총 개수 지정 (예: 250개 수집 목표)
MAX_COMMENTS = 10000
progress_bar = tqdm(total=MAX_COMMENTS, desc="댓글 수집 중", unit="개")

while True:
    for i in range(1, 26):  # 댓글+화수 쌍 최대 25개까지
        try:
            comment_xpath = f'//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div[3]/div/div[2]/div[{i}]/div/div[1]/div[2]/div[2]/div[1]/span[1]'
            episode_xpath = f'//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div[3]/div/div[2]/div[{i}]/div/div[1]/div[2]/div[2]/div[1]/span[2]'

            comment = driver.find_element(By.XPATH, comment_xpath).text.strip()
            episode = driver.find_element(By.XPATH, episode_xpath).text.strip()

            results.append([comment, episode])
            progress_bar.update(1)
            
            if len(results) >= MAX_COMMENTS:
                break
                
        except NoSuchElementException:
            continue

    if len(results) >= MAX_COMMENTS:
        break
            
    # 다음 페이지 버튼 찾기
    try:
        next_page_index = len(results) + 1  # 대략적인 위치 추정
        next_button_xpath = f'//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div[3]/div/div[2]/div[{next_page_index}]'

        next_button = driver.find_element(By.XPATH, next_button_xpath)
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()
        time.sleep(2)
    except NoSuchElementException:
        print("더 이상 페이지 없음.")
        break

progress_bar.close()  # ✅ 프로그레스바 닫기        
        
# CSV 저장
with open('kakao_comments1.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['댓글', '화수'])
    writer.writerows(results)

driver.quit()
print("CSV 저장 완료")

댓글 수집 중:  81%|█████████████████████████████████████████████████▎           | 8075/10000 [2:30:13<35:48,  1.12s/개]

더 이상 페이지 없음.


CSV 저장 완료
